<a href="https://colab.research.google.com/github/brendanpshea/data-science/blob/main/DataScience_05_WriteBetterQueries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Chapter 5: Writing Better Queries

When dealing with large amounts of data, the ability to write effective queries is not just a skill—it's an art form that can significantly impact the success of your projects. **Efficient queries** are the cornerstone of data analysis, enabling you to extract meaningful insights from vast seas of information with precision and speed.

But why does "better" matter when it comes to writing queries? The answer lies in the **three critical factors** that define the quality of your data interactions:

1. *Performance*. Better queries run faster, consuming less computational resources. In a world where time is money, this translates to cost savings and quicker insights.

2. *Accuracy*. Improved query writing ensures you're extracting exactly the data you need. This precision minimizes errors and misinterpretations that could lead to flawed analyses.

3. *Scalability*. As your data grows, well-crafted queries continue to perform efficiently, allowing your analyses to scale seamlessly.

Throughout this chapter, we'll explore techniques to enhance your query writing skills, focusing on data manipulation, optimization strategies, and best practices. By the end, you'll be equipped to craft queries that not only retrieve data but do so with elegance and efficiency.

Remember, in the world of big data, the difference between a good query and a great one can be the difference between drowning in information and surfing the waves of insight. Let's dive in and learn how to write better queries!


## Sample Data Set: Zombie Attacks!
For this chapter, we'll be dealing with data set about zombie attacks. Let's start by loading our data set and taking a look.

In [1]:
!wget https://github.com/brendanpshea/data-science/raw/main/data/zombie_attacks.csv -q -nc

In [2]:
## load csv file into a sqlite database
import pandas as pd
import sqlite3

# Load the CSV file into a DataFrame
df = pd.read_csv('zombie_attacks.csv')

# Save to SQLite
conn = sqlite3.connect('zombie_attacks.db')
df.to_sql('ZombieAttacks', conn, if_exists='replace', index=False)
conn.close()

### Getting to Know Our Data
Now, let's connect to the database and take a look at our data.

In [3]:
%reload_ext sql
%config SqlMagic.autopandas = True
%sql sqlite:///zombie_attacks.db

In [4]:
%%sql
--Get table schema
PRAGMA table_info(ZombieAttacks);

 * sqlite:///zombie_attacks.db
Done.


,cid,name,type,notnull,dflt_value,pk
0,0,Date,TEXT,0,None,0
1,1,Location,TEXT,0,None,0
2,2,ZombieType,TEXT,0,None,0
3,3,VictimCount,REAL,0,None,0
4,4,SurvivalRate,REAL,0,None,0
5,5,WeatherCondition,TEXT,0,None,0
6,6,MoonPhase,TEXT,0,None,0
7,7,TemperatureCelsius,REAL,0,None,0
8,8,HumidityPercent,REAL,0,None,0
9,9,WindSpeedKmh,REAL,0,None,0


In [5]:
%%sql
SELECT *
FROM ZombieAttacks
LIMIT 5;

 * sqlite:///zombie_attacks.db
Done.


,Date,Location,ZombieType,VictimCount,SurvivalRate,WeatherCondition,MoonPhase,TemperatureCelsius,HumidityPercent,WindSpeedKmh,PopulationDensity,EmergencyResponseTime,Month
0,2023-02-24,Des Moines,Runner,25.5,0.2215,Foggy,Full Moon,12.8,58.9,8.1,65.1,8.7,2.0
1,2023-09-29,Rochester,Walker,13.0,0.3739,Stormy,Waxing Crescent,21.2,46.4,35.6,594.5,4.0,9.0
2,2023-06-01,Rochester,Walker,12.0,0.1924,Cloudy,New Moon,20.6,31.9,9.9,236.4,13.2,6.0
3,2023-02-14,St. Louis,Crawler,7.0,0.7949,Stormy,New Moon,10.6,57.9,5.3,156.8,4.7,2.0
4,2023-08-31,Winnipeg,Runner,39.0,0.0678,Sunny,Full Moon,16.7,45.1,3.8,19.6,6.9,8.0


### Data Dictionary for `zombie_attacks.csv`

| **Column Name** | **Data Type** | **Description** |
| --- | --- | --- |
| `Date` | `datetime` | The date of the recorded zombie attack. |
| `Location` | `string` | The location where the zombie attack occurred, centered around major cities near Minneapolis. |
| `ZombieType` | `string` | The type of zombie involved in the attack, with possible values: 'Walker', 'Runner', 'Crawler', 'Jumper'. |
| `VictimCount` | `integer` | The number of victims in the zombie attack. |
| `SurvivalRate` | `float` | The survival rate of victims, represented as a proportion between 0 and 1. |
| `WeatherCondition` | `string` | The weather condition at the time of the attack, with possible values: 'Sunny', 'Rainy', 'Cloudy', 'Foggy', 'Stormy'. |
| `MoonPhase` | `string` | The phase of the moon at the time of the attack, with possible values: 'New Moon', 'Waxing Crescent', 'First Quarter', 'Waxing Gibbous', 'Full Moon', 'Waning Gibbous', 'Last Quarter', 'Waning Crescent'. |
| `TemperatureCelsius` | `float` | The temperature in degrees Celsius at the time of the attack, adjusted for weather conditions and location-specific patterns. |
| `HumidityPercent` | `float` | The humidity percentage at the time of the attack. |
| `WindSpeedKmh` | `float` | The wind speed in kilometers per hour at the time of the attack. |
| `PopulationDensity` | `float` | The population density of the location where the attack occurred. |
| `EmergencyResponseTime` | `float` | The time in minutes for emergency response to arrive at the scene of the attack. |
| `Month` | `integer` | The month of the year when the attack occurred, extracted from the `Date` column. |

## Filtering Data

Filtering is a fundamental operation in data analysis, allowing you to extract specific subsets of data based on certain conditions. In SQL, filtering is primarily done using the `WHERE` clause. Let's explore various filtering techniques using our Zombie Attacks dataset.

### Basic Filtering

The simplest form of filtering uses comparison operators.

**Example: Find all zombie attacks with more than 20 victims**

In [6]:
%%sql
SELECT
    Date,
    Location,
    ZombieType,
    VictimCount
FROM ZombieAttacks
WHERE VictimCount > 20
LIMIT 5;

 * sqlite:///zombie_attacks.db
Done.


,Date,Location,ZombieType,VictimCount
0,2023-02-24,Des Moines,Runner,25.5
1,2023-08-31,Winnipeg,Runner,39.0
2,2023-01-03,Chicago,Walker,25.0
3,2023-03-08,None,Walker,23.0
4,2023-01-08,Chicago,Runner,28.5


This query returns all attacks where the victim count exceeds 20. Other comparison operators include `<` (less than), `=` (equal to), `>=` (greater than or equal to), `<=` (less than or equal to), and `<>` (not equal to).

#### Filtering with Multiple Conditions

You can combine multiple conditions using logical operators like `AND`, `OR`, and `NOT`.

**Example: Find Runner zombies in Rochester with more than 10 victims**

In [7]:
%%sql
SELECT
    Date,
    Location,
    ZombieType,
    VictimCount
FROM ZombieAttacks
WHERE ZombieType = 'Runner'
  AND Location = 'Rochester'
  AND VictimCount > 10
LIMIT 5;

 * sqlite:///zombie_attacks.db
Done.


,Date,Location,ZombieType,VictimCount
0,2023-11-06,Rochester,Runner,19.5
1,2023-06-22,Rochester,Runner,19.5
2,2023-06-02,Rochester,Runner,18.0
3,2023-08-08,Rochester,Runner,10.5
4,2023-11-15,Rochester,Runner,24.0


This query demonstrates the use of `AND` to combine multiple conditions.

#### Filtering with IN Clause

The `IN` clause is useful when you want to match against multiple possible values.

**Example: Find attacks in either Rochester, Minneapolis, or Madison**

In [8]:
%%sql
SELECT
    Date,
    Location,
    ZombieType,
    VictimCount
FROM ZombieAttacks
WHERE Location IN ('Rochester', 'Minneapolis', 'Madison')
LIMIT 5;

 * sqlite:///zombie_attacks.db
Done.


,Date,Location,ZombieType,VictimCount
0,2023-09-29,Rochester,Walker,13.0
1,2023-06-01,Rochester,Walker,12.0
2,2023-08-06,Minneapolis,Walker,6.0
3,2023-02-21,Rochester,Walker,10.0
4,2023-01-16,Minneapolis,Walker,6.0


This query retrieves attacks from any of the specified cities.

#### Filtering with LIKE Clause

The `LIKE` clause is used for pattern matching in string fields.

**Example: Find all attacks where the weather condition includes the word "Rain"**

In [9]:
%%sql
SELECT
    Date,
    Location,
    ZombieType,
    WeatherCondition
FROM ZombieAttacks
WHERE WeatherCondition LIKE '%Rain%'
LIMIT 5;

 * sqlite:///zombie_attacks.db
Done.


,Date,Location,ZombieType,WeatherCondition
0,2023-03-06,Kansas City,Crawler,Rainy
1,2023-02-21,Rochester,Walker,Rainy
2,2023-09-10,Milwaukee,Walker,Rainy
3,2023-12-03,Des Moines,Walker,Rainy
4,2023-05-28,Minneapolis,Runner,Rainy


This query will match 'Rainy', 'Light Rain', 'Heavy Rain', etc. The `%` is a wildcard that matches any number of characters.

#### Filtering with BETWEEN Clause

`BETWEEN` is used to filter values within a range.

**Example: Find attacks with temperatures between 15°C and 25°C**

In [10]:
%%sql
SELECT
    Date,
    Location,
    ZombieType,
    TemperatureCelsius
FROM ZombieAttacks
WHERE TemperatureCelsius BETWEEN 15 AND 25
LIMIT 5;

 * sqlite:///zombie_attacks.db
Done.


,Date,Location,ZombieType,TemperatureCelsius
0,2023-09-29,Rochester,Walker,21.2
1,2023-06-01,Rochester,Walker,20.6
2,2023-08-31,Winnipeg,Runner,16.7
3,2023-04-24,Chicago,Walker,17.4
4,2023-03-06,Kansas City,Crawler,24.0


This query retrieves all attacks that occurred when the temperature was between 15°C and 25°C, inclusive.

#### Filtering Null Values

Sometimes, you need to filter based on the presence or absence of data.

**Example: Find attacks where the emergency response time is not recorded**

In [11]:
%%sql
SELECT
    Date,
    Location,
    ZombieType,
    EmergencyResponseTime
FROM ZombieAttacks
WHERE EmergencyResponseTime IS NULL
LIMIT 5;

 * sqlite:///zombie_attacks.db
Done.


,Date,Location,ZombieType,EmergencyResponseTime
0,2023-10-23,Chicago,Walker,None
1,2023-01-21,Madison,Crawler,None
2,2023-05-16,Minneapolis,Jumper,None
3,2023-08-28,Fargo,Runner,None
4,2023-02-25,Fargo,Runner,None


Use `IS NULL` to find rows where a column has no value, and `IS NOT NULL` to find rows where a column has any value.

#### Filtering with Subqueries

Subqueries allow you to use the result of one query to filter another.

**Example: Find attacks with above-average victim counts**

In [12]:
%%sql
SELECT
    Date,
    Location,
    ZombieType,
    VictimCount
FROM ZombieAttacks
WHERE VictimCount > (SELECT AVG(VictimCount) FROM ZombieAttacks)
LIMIT 5;

 * sqlite:///zombie_attacks.db
Done.


,Date,Location,ZombieType,VictimCount
0,2023-02-24,Des Moines,Runner,25.5
1,2023-09-29,Rochester,Walker,13.0
2,2023-06-01,Rochester,Walker,12.0
3,2023-08-31,Winnipeg,Runner,39.0
4,2023-10-23,Chicago,Walker,12.0


This query first calculates the average victim count across all attacks, then returns only the attacks that exceed this average.

## Sorting Data

Sorting is a crucial operation in data analysis, allowing you to organize your query results in a specific order. In SQL, sorting is primarily done using the `ORDER BY` clause. Let's explore various sorting techniques using our Zombie Attacks dataset.

#### Basic Sorting

The simplest form of sorting arranges data based on a single column, either in ascending (ASC) or descending (DESC) order.

**Example: Sort zombie attacks by date, showing the most recent first**

In [13]:
%%sql
SELECT
    Date,
    Location,
    ZombieType,
    VictimCount
FROM ZombieAttacks
ORDER BY Date DESC
LIMIT 5;

 * sqlite:///zombie_attacks.db
Done.


,Date,Location,ZombieType,VictimCount
0,2023-12-31,Des Moines,Runner,6.0
1,2023-12-31,Des Moines,Walker,6.0
2,2023-12-31,Winnipeg,Runner,18.0
3,2023-12-30,Milwaukee,Walker,9.0
4,2023-12-30,Milwaukee,Walker,8.0


This query returns the 5 most recent zombie attacks. By default, `ORDER BY` sorts in ascending order, so we use `DESC` to get the most recent dates first.

#### Sorting by Multiple Columns

You can sort by multiple columns to create a hierarchical order.

**Example: Sort attacks by location, then by date within each location**

In [14]:
%%sql
SELECT
    Date,
    Location,
    ZombieType,
    VictimCount
FROM ZombieAttacks
WHERE Location IS NOT NULL
AND Date IS NOT NULL
ORDER BY Location, Date
LIMIT 5;

 * sqlite:///zombie_attacks.db
Done.


,Date,Location,ZombieType,VictimCount
0,2023-01-03,Chicago,Walker,25.0
1,2023-01-05,Chicago,Crawler,8.0
2,2023-01-05,Chicago,Walker,17.0
3,2023-01-07,Chicago,Walker,5.0
4,2023-01-07,Chicago,Runner,7.5


This query first sorts the attacks alphabetically by location, and then within each location, it sorts by date in descending order.

#### Sorting with Expressions

You can use expressions in the `ORDER BY` clause to sort based on computed values.

**Example: Sort attacks by survival rate, considering the victim count**

In [15]:
%%sql
SELECT
    Date,
    Location,
    ZombieType,
   (1 - SurvivalRate) * VictimCount AS Casualties
FROM ZombieAttacks
ORDER BY Casualties DESC
LIMIT 5;

 * sqlite:///zombie_attacks.db
Done.


,Date,Location,ZombieType,Casualties
0,2023-08-31,Winnipeg,Runner,36.35580
1,2023-08-07,Minneapolis,Runner,31.07070
2,2023-07-19,Winnipeg,Runner,29.99745
3,2023-04-12,Des Moines,Runner,29.11545
4,2023-01-08,Chicago,Runner,26.78430


This query calculates a 'Casualties' value and sorts the results based on this calculated field, showing the 5 attacks with the highest casualty counts.

#### Sorting with Case Statements

Case statements in the `ORDER BY` clause allow for complex, conditional sorting.

**Example: Sort zombie types in a specific order, then by date**

In [16]:
%%sql
SELECT
    Date,
    Location,
    ZombieType,
    VictimCount
FROM ZombieAttacks
ORDER BY
    CASE ZombieType
        WHEN 'Runner' THEN 1
        WHEN 'Jumper' THEN 2
        WHEN 'Walker' THEN 3
        WHEN 'Crawler' THEN 4
        ELSE 5
    END,
    Date DESC
LIMIT 5;

 * sqlite:///zombie_attacks.db
Done.


,Date,Location,ZombieType,VictimCount
0,2023-12-31,Des Moines,Runner,6.0
1,2023-12-31,Winnipeg,Runner,18.0
2,2023-12-25,Des Moines,Runner,18.0
3,2023-12-24,Des Moines,Runner,9.0
4,2023-12-22,Fargo,Runner,25.5


This query sorts the attacks first by a custom zombie type order, and then by date within each type.

#### Sorting with Nulls

By default, NULL values are sorted differently in different database systems. You can control their position in the sorted results.

**Example: Sort attacks by emergency response time, with missing values last**

In [17]:
%%sql
SELECT
    Date,
    Location,
    ZombieType,
    EmergencyResponseTime
FROM ZombieAttacks
ORDER BY EmergencyResponseTime DESC NULLS LAST
LIMIT 5;

 * sqlite:///zombie_attacks.db
Done.


,Date,Location,ZombieType,EmergencyResponseTime
0,2023-09-20,Des Moines,Runner,50.7
1,2023-07-07,Madison,Runner,42.4
2,2023-03-04,Winnipeg,Crawler,39.9
3,2023-04-28,Minneapolis,Crawler,38.9
4,2023-09-12,Madison,Jumper,35.5


This query ensures that attacks with no recorded emergency response time appear at the end of the sorted list.

### Date Functions

Working with dates and times is a common task in data analysis. SQLite, despite its lightweight nature, provides several useful functions for handling date and time data. Let's explore some of the basic date functions in SQLite that are particularly useful for beginners.

#### Basic SQLite Date Functions

**date(timestring, modifier)**: This function returns the date in the format 'YYYY-MM-DD'. Example:

In [18]:
%%sql
SELECT date('now');

 * sqlite:///zombie_attacks.db
Done.


,date('now')
0,2024-07-06


**time(timestring, modifier)**: This function returns the time in the format 'HH:MM:SS'. Example:

In [19]:
%%sql
SELECT time('now');

 * sqlite:///zombie_attacks.db
Done.


,time('now')
0,12:20:57


**datetime(timestring, modifier)**: This function returns the date and time in the format 'YYYY-MM-DD HH:MM:SS'. Example:

In [20]:
%%sql
SELECT datetime('now');

 * sqlite:///zombie_attacks.db
Done.


,datetime('now')
0,2024-07-06 12:20:57


**strftime(format, timestring, modifier)**: This versatile function allows you to format date and time in various ways. Example:

In [21]:
%%sql
-- in US style
SELECT strftime('%m-%d-%Y', 'now');

 * sqlite:///zombie_attacks.db
Done.


,"strftime('%m-%d-%Y', 'now')"
0,07-06-2024


In [22]:
%%sql
-- spell out months
SELECT strftime('%B %d, %Y', 'now');

 * sqlite:///zombie_attacks.db
Done.


,"strftime('%B %d, %Y', 'now')"
0,None


These functions can be used with our Zombie Attacks database. For instance, to get the day of the week for each attack:

In [23]:
%%sql
SELECT
    Date,
    Location,
    ZombieType,
    VictimCount,
    strftime('%w', Date) AS DayOfWeek
FROM ZombieAttacks
LIMIT 5;

 * sqlite:///zombie_attacks.db
Done.


,Date,Location,ZombieType,VictimCount,DayOfWeek
0,2023-02-24,Des Moines,Runner,25.5,5
1,2023-09-29,Rochester,Walker,13.0,5
2,2023-06-01,Rochester,Walker,12.0,4
3,2023-02-14,St. Louis,Crawler,7.0,2
4,2023-08-31,Winnipeg,Runner,39.0,4


#### Modifiers in SQLite Date Functions

SQLite date functions also accept modifiers that allow you to perform date arithmetic. Some common modifiers include:

-   '+N days'
-   '-N days'
-   '+N months'
-   '+N years'
-   'start of month'
-   'start of year'

For example, to get the date 7 days from now:

In [24]:
%%sql
SELECT date('now', '+7 days');

 * sqlite:///zombie_attacks.db
Done.


,"date('now', '+7 days')"
0,2024-07-13


Or ten years, 2 months, and 3 weeks from now.

In [25]:
%%sql
SELECT date('now', '+10 years', '+2 months', '+21 days');

 * sqlite:///zombie_attacks.db
Done.


,"date('now', '+10 years', '+2 months', '+21 days')"
0,2034-09-27


#### Comparison with Other Databases

While these functions are specific to SQLite, other database systems have their own ways of handling dates. Here's a comparison table showing equivalent operations in SQLite with PostgreSQL and MySQL (two widely used open source relational databases):

| Operation | SQLite | PostgreSQL | MySQL |
| --- | --- | --- | --- |
| Current Date | `date('now')` | `CURRENT_DATE` | `CURDATE()` |
| Current Time | `time('now')` | `CURRENT_TIME` | `CURTIME()` |
| Current Date and Time | `datetime('now')` | `CURRENT_TIMESTAMP` | `NOW()` |
| Extract Year | `strftime('%Y', date)` | `EXTRACT(YEAR FROM date)` | `YEAR(date)` |
| Extract Month | `strftime('%m', date)` | `EXTRACT(MONTH FROM date)` | `MONTH(date)` |
| Extract Day | `strftime('%d', date)` | `EXTRACT(DAY FROM date)` | `DAY(date)` |
| Format Date | `strftime('%Y-%m-%d', date)` | `TO_CHAR(date, 'YYYY-MM-DD')` | `DATE_FORMAT(date, '%Y-%m-%d')` |
| Add Days | `date(date, '+N days')` | `date + INTERVAL 'N days'` | `DATE_ADD(date, INTERVAL N DAY)` |
| Subtract Days | `date(date, '-N days')` | `date - INTERVAL 'N days'` | `DATE_SUB(date, INTERVAL N DAY)` |

This table provides a quick reference for equivalent date operations across these three popular database systems. While the specific syntax may differ, the general concepts remain the same.

### Logical Functions

Logical functions in SQL allow you to perform conditional operations, create categories, and handle complex decision-making within your queries. These functions are essential for data analysis and reporting. Let's explore various logical functions using our Zombie Attacks dataset.

#### CASE Statements

The CASE statement is one of the most versatile logical functions in SQL, allowing for multiple conditional checks.

**Example: Categorize attacks based on victim count**

In [26]:
%%sql
SELECT
    Date,
    Location,
    ZombieType,
    VictimCount,
    CASE
        WHEN VictimCount < 10 THEN 'Low'
        WHEN VictimCount BETWEEN 10 AND 50 THEN 'Medium'
        WHEN VictimCount > 50 THEN 'High'
        ELSE 'Unknown'
    END as AttackSeverity
FROM ZombieAttacks
LIMIT 5;

 * sqlite:///zombie_attacks.db
Done.


,Date,Location,ZombieType,VictimCount,AttackSeverity
0,2023-02-24,Des Moines,Runner,25.5,Medium
1,2023-09-29,Rochester,Walker,13.0,Medium
2,2023-06-01,Rochester,Walker,12.0,Medium
3,2023-02-14,St. Louis,Crawler,7.0,Low
4,2023-08-31,Winnipeg,Runner,39.0,Medium


This query adds a new column categorizing attacks as Low, Medium, or High based on the victim count.

#### IF/ELSE Logic

While not all SQL databases support a direct IF function, you can achieve similar results with a simple CASE statement.

**Example: Flag attacks with above-average victim counts**

In [27]:
%%sql
SELECT
    Date,
    Location,
    ZombieType,
    VictimCount,
    IIF (VictimCount > (SELECT AVG(VictimCount) FROM ZombieAttacks), 'Above Average', 'Below Average') AS AttackType
FROM ZombieAttacks
LIMIT 5;

 * sqlite:///zombie_attacks.db
Done.


,Date,Location,ZombieType,VictimCount,AttackType
0,2023-02-24,Des Moines,Runner,25.5,Above Average
1,2023-09-29,Rochester,Walker,13.0,Above Average
2,2023-06-01,Rochester,Walker,12.0,Above Average
3,2023-02-14,St. Louis,Crawler,7.0,Below Average
4,2023-08-31,Winnipeg,Runner,39.0,Above Average


Here, use the `IIF` function to do the following:

1. We check whether Victim count is higher than average.
2. IF this is true, we return "above average".
3. ELSE (if it is lower than average), we return "below average".

This query compares each attack's victim count to the overall average, categorizing it as either 'Above Average' or 'Below Average'.

#### COALESCE Function

COALESCE returns the first non-null expression in a list. It's useful for handling null values or providing default values.

**Example: Provide a default value for missing emergency response times**

In [28]:
%%sql
SELECT
    Date,
    Location,
    ZombieType,
    VictimCount,
    EmergencyResponseTime,
    COALESCE(EmergencyResponseTime, 30) as AdjustedResponseTime
FROM ZombieAttacks
LIMIT 10;

 * sqlite:///zombie_attacks.db
Done.


,Date,Location,ZombieType,VictimCount,EmergencyResponseTime,AdjustedResponseTime
0,2023-02-24,Des Moines,Runner,25.5,8.7,8.7
1,2023-09-29,Rochester,Walker,13.0,4.0,4.0
2,2023-06-01,Rochester,Walker,12.0,13.2,13.2
3,2023-02-14,St. Louis,Crawler,7.0,4.7,4.7
4,2023-08-31,Winnipeg,Runner,39.0,6.9,6.9
5,2023-10-23,Chicago,Walker,12.0,NaN,30.0
6,2023-05-27,Kansas City,Walker,3.0,3.9,3.9
7,2023-08-06,Minneapolis,Walker,6.0,3.3,3.3
8,2023-04-24,Chicago,Walker,10.0,6.1,6.1
9,2023-03-06,Kansas City,Crawler,9.0,6.8,6.8


This query replaces any NULL values in EmergencyResponseTime with a default value of 30.

#### Boolean Logic

SQL supports boolean operators like AND, OR, and NOT for complex conditional logic.

**Example: Find high-severity attacks in warm weather**

In [29]:
%%sql
SELECT
    Date,
    Location,
    ZombieType,
    TemperatureCelsius,
    WeatherCondition
FROM ZombieAttacks
WHERE
    VictimCount > 20
    AND TemperatureCelsius > 20
    AND WeatherCondition NOT IN ('Rainy', 'Stormy')
LIMIT 5;

 * sqlite:///zombie_attacks.db
Done.


,Date,Location,ZombieType,TemperatureCelsius,WeatherCondition
0,2023-04-12,Kansas City,Runner,22.3,Foggy
1,2023-12-22,Fargo,Runner,35.4,Sunny
2,2023-06-18,Chicago,Runner,20.9,Foggy
3,2023-08-26,Milwaukee,Runner,30.4,Sunny
4,2023-07-19,Winnipeg,Runner,24.0,Foggy


This query combines multiple conditions to find specific types of attacks.

#### Logical Functions in Aggregations

Logical functions can be used within aggregations for more complex analyses.

**Example: Calculate the percentage of attacks by runners in each location**

In [30]:
%%sql
SELECT
    Location,
    COUNT(*) as TotalAttacks,
    SUM(CASE WHEN ZombieType = 'Runner' THEN 1 ELSE 0 END) as RunnerAttacks,
    ROUND(100.0 * SUM(CASE WHEN ZombieType = 'Runner' THEN 1 ELSE 0 END) / COUNT(*), 2) as PercentRunnerAttacks
FROM ZombieAttacks
GROUP BY Location
ORDER BY PercentRunnerAttacks DESC
LIMIT 5;

 * sqlite:///zombie_attacks.db
Done.


,Location,TotalAttacks,RunnerAttacks,PercentRunnerAttacks
0,Des Moines,90,35,38.89
1,Fargo,103,34,33.01
2,St. Louis,124,38,30.65
3,Winnipeg,109,32,29.36
4,Kansas City,92,27,29.35


This query uses a CASE statement within a SUM function to calculate percentages.

### System Functions

SQLite (like other relational databases) provides a set of built-in functions that can be considered system functions. These functions provide information about the database, the current state of operations, and perform system-related tasks. Let's explore some of the key SQLite system functions:

#### SQLite Version Information

This function provides information about the SQLite version in use.

**Example: Retrieve SQLite version**

In [31]:
%%sql
SELECT sqlite_version();

 * sqlite:///zombie_attacks.db
Done.


,sqlite_version()
0,3.37.2


This query returns the version of SQLite being used.

#### Database Last Insert Information

These functions provide information about the last insert operation.

**Example: Get information about the last insert**

In [32]:
%%sql
INSERT INTO ZombieAttacks (Date, Location, ZombieType)
VALUES (CURRENT_TIMESTAMP, 'New York', 'Walker');

SELECT
    last_insert_rowid() AS LastInsertedID,
    changes() AS RowsAffected;

 * sqlite:///zombie_attacks.db
1 rows affected.
Done.


,LastInsertedID,RowsAffected
0,1001,1


This query inserts a new row into the ZombieAttacks table and then retrieves the ID of the last inserted row and the number of rows affected by the last insert, update, or delete operation.

#### Database and Table Information

SQLite provides pragmas (special commands) to retrieve system information about the database and its tables.

**Example: Get information about the ZombieAttacks table**

In [33]:
%%sql
PRAGMA table_info(ZombieAttacks);

 * sqlite:///zombie_attacks.db
Done.


,cid,name,type,notnull,dflt_value,pk
0,0,Date,TEXT,0,None,0
1,1,Location,TEXT,0,None,0
2,2,ZombieType,TEXT,0,None,0
3,3,VictimCount,REAL,0,None,0
4,4,SurvivalRate,REAL,0,None,0
5,5,WeatherCondition,TEXT,0,None,0
6,6,MoonPhase,TEXT,0,None,0
7,7,TemperatureCelsius,REAL,0,None,0
8,8,HumidityPercent,REAL,0,None,0
9,9,WindSpeedKmh,REAL,0,None,0


#### Database Statistics

SQLite provides a way to get statistics about the database, though it requires a special table.

**Example: Get database statistics**

In [34]:
%%sql
ANALYZE ZombieAttacks;

SELECT * FROM sqlite_stat1;

 * sqlite:///zombie_attacks.db
Done.
Done.


,tbl,idx,stat
0,ZombieAttacks,None,1001


Right now, this doesn't tell us much:

- `tbl = ZombieAttacks` ndicates that the statistics are for the ZombieAttacks table.
- `idx = None` suggests that the statistics are for the entire ZombieAttacks table, not for a specific index.
`stat = 1001`  represents the number of entries (or rows) in the ZombieAttacks table.

Statistics like these become more valuable as we build more tables (and indexes on these tables).

#### Random Number Generation

While not strictly a "system" function, SQLite's random number generator is often used in system-level operations.

**Example: Generate random numbers of different types**

In [35]:
%%sql
SELECT
    RANDOM() AS RandomNumber;

 * sqlite:///zombie_attacks.db
Done.


,RandomNumber
0,8634902770870816089


This query generates a random integer between -9223372036854775808 and +9223372036854775807. We can use to do things such as randomly sample rows from our database.

In [36]:
%%sql
SELECT
    Date,
    Location,
    ZombieType,
    VictimCount
FROM ZombieAttacks
ORDER BY RANDOM()
LIMIT 5;

 * sqlite:///zombie_attacks.db
Done.


,Date,Location,ZombieType,VictimCount
0,2023-11-21,St. Louis,Runner,13.5
1,2023-10-20,Rochester,Jumper,4.0
2,2023-06-26,St. Louis,Walker,13.0
3,2023-01-23,Minneapolis,Crawler,6.0
4,2023-01-03,Rochester,Walker,9.0


This query allows us to **randomly sample** a subset of zombie encounters, which plays a role in many statistical methods.

#### System functions in other databases
Since system functions aren't part of ANSI-standard SQL, the exact syntax varies a bit by database. For example:

| Function Type | SQLite | PostgreSQL | MySQL |
| --- | --- | --- | --- |
| Version | `sqlite_version()` | `version()` | `VERSION()` |
| Current User | N/A | `current_user` | `CURRENT_USER()` |
| Current Database | N/A | `current_database()` | `DATABASE()` |
| Last Insert ID | `last_insert_rowid()` | `lastval()` | `LAST_INSERT_ID()` |
| Affected Rows | `changes()` | `pg_affected_rows()` | `ROW_COUNT()` |
| Current Date/Time | `datetime('now')` | `current_timestamp` | `NOW()` |
| Random Number | `random()` | `random()` | `RAND()` |

This table illustrates that while the concept of system functions is common across

## Query Optimization
-
Query optimization is the process of improving the performance of SQL queries. A well-optimized query can dramatically reduce execution time and resource usage, which is crucial when working with large datasets or in systems with many concurrent users. One of the fundamental techniques in query optimization is parameterization.

## Parameterization

Parameterization, also known as query parameterization or prepared statements, is a powerful technique used to optimize the execution of repetitive queries. At its core, parameterization involves replacing hard-coded values in SQL statements with placeholders, called parameters. These parameters are then filled with actual values when the query is executed.

To understand the importance of parameterization, we need to first look at how databases typically process queries. When a database receives a query, it goes through several stages:

1.  **Parsing.** The database breaks down the SQL statement into a tree-like structure.
2.  **Optimization.** The database determines the most efficient way to execute the query.
3.  **Execution Plan Generation.** Based on the optimization, an execution plan is created.
4.  **Execution.** The plan is carried out and results are returned.

Parameterization leverages the fact that many queries in a typical application are structurally identical, differing only in the specific values they use. By using parameters instead of hard-coded values, we allow the database to recognize these structural similarities. This recognition enables a crucial optimization: the database can cache the execution plan after the first execution of a parameterized query. When the same query structure is encountered again, even with different parameter values, the database can reuse this cached plan, skipping the time-consuming parsing and optimization steps.

Paramterization is also crucially important in preventing **SQL Injection Attacks**, which can occur when "unsanitized input" (for example, what a user types into a website in the "username" and "password") directly into a database. (For example: a hacker types damaging SQL code instead of their username).

Let's look at a concrete example using Python to connect to our Zombie Attacks SQLite database:

In [37]:
import sqlite3

def get_zombie_attacks(location, zombie_type):
    conn = sqlite3.connect('zombie_attacks.db')
    cursor = conn.cursor()

    # Non-parameterized query (vulnerable to SQL injection)
    # query = f"SELECT * FROM ZombieAttacks WHERE Location = '{location}' AND ZombieType = '{zombie_type}'"

    # Parameterized query
    query = "SELECT * FROM ZombieAttacks WHERE Location = ? AND ZombieType = ? LIMIT 5"

    cursor.execute(query, (location, zombie_type))
    results = cursor.fetchall()

    conn.close()
    return results

# Usage
attacks = get_zombie_attacks('Rochester', 'Runner')
for attack in attacks:
    print(attack)

('2023-11-06', 'Rochester', 'Runner', 19.5, 0.2593, 'Cloudy', 'Waxing Crescent', 12.0, 74.5, 2.2, 98.9, 13.6, 11.0)
('2023-06-22', 'Rochester', 'Runner', 19.5, 0.1351, 'Foggy', 'Full Moon', 11.8, 73.4, 6.5, 1154.0, 13.5, 6.0)
('2023-10-23', 'Rochester', 'Runner', 7.5, 0.4707, 'Foggy', 'New Moon', 19.9, 50.3, 13.5, 453.0, 8.1, 10.0)
('2023-06-02', 'Rochester', 'Runner', 18.0, 0.6397, 'Stormy', 'First Quarter', 22.5, 38.7, 2.2, 196.6, 14.7, 6.0)
('2023-08-08', 'Rochester', 'Runner', 10.5, 0.4552, 'Cloudy', 'Full Moon', 7.3, 43.9, 3.8, 206.6, 6.8, 8.0)


###  Indexing

Indexing is a fundamental technique in query optimization that can dramatically improve the speed of data retrieval operations. An index in a database is analogous to an index in a book: it allows the database engine to quickly locate the data it needs without having to scan every row in a table.

At its core, an **index** is a data structure that stores a subset of a table's data in a way that's optimized for quick searches. Typically, an index contains one or more columns from a table, along with pointers to the full rows in the table. When a query is executed that can use the index, the database engine can quickly narrow down the set of rows it needs to examine, rather than scanning the entire table.

The most common type of index is a **B-tree index**, which organizes data in a tree-like structure that allows for efficient searching, insertion, and deletion operations. However, other types of indexes exist, such as bitmap indexes or hash indexes, each with their own strengths for different types of data and query patterns.

While indexes can significantly speed up data retrieval, they come with trade-offs. Each index takes up additional storage space, and writes to the table (inserts, updates, and deletes) become slightly slower because the indexes must be updated along with the table data. Therefore, the decision to add an index should be based on a careful analysis of your query patterns and data usage.

Let's consider our Zombie Attacks database. Suppose we frequently query the database to find attacks in specific locations or by specific zombie types. We might decide to create indexes on the Location and ZombieType columns:

In [38]:
%%sql
CREATE INDEX idx_location ON ZombieAttacks(Location);
CREATE INDEX idx_zombie_type ON ZombieAttacks(ZombieType);

 * sqlite:///zombie_attacks.db
Done.
Done.


""


After creating these indexes, queries that filter on Location or ZombieType can potentially run much faster. For example:

In [40]:
%%sql
SELECT * FROM ZombieAttacks WHERE Location = 'Des Moines' LIMIT 5;

 * sqlite:///zombie_attacks.db
Done.


,Date,Location,ZombieType,VictimCount,SurvivalRate,WeatherCondition,MoonPhase,TemperatureCelsius,HumidityPercent,WindSpeedKmh,PopulationDensity,EmergencyResponseTime,Month
0,2023-02-24,Des Moines,Runner,25.5,0.2215,Foggy,Full Moon,12.8,58.9,8.1,65.1,8.7,2.0
1,2023-12-03,Des Moines,Walker,13.0,0.3947,Rainy,New Moon,27.2,38.0,13.8,92.9,9.0,12.0
2,2023-12-09,Des Moines,Runner,13.5,0.1715,Rainy,Waxing Gibbous,30.9,53.0,6.6,1507.8,14.9,12.0
3,2023-08-29,Des Moines,Runner,18.0,0.3913,Rainy,New Moon,35.8,38.3,4.9,177.1,10.1,8.0
4,2023-11-10,Des Moines,Runner,16.5,0.5300,Stormy,Waning Gibbous,30.7,59.9,6.9,502.3,12.3,11.0


With the idx_location index, the database can quickly locate all rows with the Location 'Des Moines' without needing to scan the entire table.

Indexes can also be created on multiple columns, which can be particularly useful for queries that frequently filter on or join using those columns together:

In [41]:
%%sql
CREATE INDEX idx_location_date ON ZombieAttacks(Location, Date);

 * sqlite:///zombie_attacks.db
Done.


""


This multi-column index would be beneficial for queries like:

In [47]:
%%sql
SELECT
    Date,
    Location,
    ZombieType,
    VictimCount
FROM ZombieAttacks
WHERE Location = 'Des Moines'
AND Date BETWEEN '2023-01-01' and '2023-01-31'
LIMIT 5;

 * sqlite:///zombie_attacks.db
Done.


,Date,Location,ZombieType,VictimCount
0,2023-01-02,Des Moines,Crawler,8.0
1,2023-01-09,Des Moines,Walker,7.0
2,2023-01-12,Des Moines,Crawler,7.0
3,2023-01-22,Des Moines,Runner,10.5
4,2023-01-29,Des Moines,Crawler,5.0


It's important to note that the order of columns in a multi-column index matters. In the above example, the index will be most efficient for queries that filter on Location first, and then on Date.

While creating indexes is straightforward, deciding which indexes to create requires careful consideration. Here are some factors to consider:

1.  Analyze which columns are frequently used in WHERE clauses, JOIN conditions, and ORDER BY clauses.
2.  Columns with high **selectivity** (many distinct values) often benefit more from indexing.
3.  For tables with frequent writes, too many indexes can slow down write operations.
4.  Smaller tables might not benefit as much from indexing, as full table scans may already be fast.

Most database systems provide tools to analyze query performance and suggest potential indexes. For example, in SQLite, you can use the EXPLAIN QUERY PLAN command to see how a query is executed and whether it's using any indexes:

In [46]:
%%sql
EXPLAIN QUERY PLAN
SELECT * FROM ZombieAttacks
WHERE Location = 'Minneapolis'
AND Date > '2023-01-01';

 * sqlite:///zombie_attacks.db
Done.


,id,parent,notused,detail
0,3,0,0,SEARCH ZombieAttacks USING INDEX idx_location_...


This command will show you the query plan, including which indexes (if any) are being used.

In the end, indexing is a powerful tool for query optimization, but it requires a thoughtful approach. By understanding your data, analyzing your query patterns, and carefully choosing your indexes, you can significantly improve the performance of your database queries. As we continue to explore query optimization techniques, remember that indexing often works in concert with other strategies to achieve the best possible query performance.

## Temporary Tables and Common Table Expressions (CTEs)

When dealing with complex queries, two powerful techniques can help improve performance and readability: Temporary Tables and Common Table Expressions (CTEs). Both serve to break down complex operations into more manageable steps, but they do so in different ways and with different trade-offs.

#### Temporary Tables

**Temporary tables** are database objects that exist temporarily within a database session and are automatically dropped when the session ends. They serve as a workspace for intermediate results, allowing for more efficient and readable queries, especially when dealing with complex data manipulations.

The primary purpose of temporary tables is to store intermediate results, which can reduce the need for subqueries or complex joins. This approach not only can improve query performance but also makes the query logic easier to understand and maintain.

Here's an example using our Zombie Attacks database:

In [ ]:
%%sql
-- Create a temporary table of high-impact attacks
CREATE TEMPORARY TABLE HighImpactAttacks AS
SELECT * FROM ZombieAttacks
WHERE VictimCount > 20;

-- Now use this temporary table for further analysis
SELECT Location, COUNT(*) as AttackCount, AVG(VictimCount) as AvgVictims
FROM HighImpactAttacks
GROUP BY Location
ORDER BY AttackCount DESC;

In this example, we first create a temporary table `HighImpactAttacks` that contains only the attacks with more than 20 victims. We then use this temporary table to calculate statistics for each location.

#### Common Table Expressions (CTEs)

CTEs, introduced with the SQL:1999 standard, provide a way to write auxiliary statements for use in a larger query. Unlike temporary tables, CTEs exist only for the duration of the query. They're defined using a WITH clause at the beginning of a SELECT, INSERT, UPDATE, DELETE, or MERGE statement.

Here's the same example using a CTE instead of a temporary table:

In [ ]:
%%sql
WITH HighImpactAttacks AS (
    SELECT * FROM ZombieAttacks
    WHERE VictimCount > 100
)
SELECT Location, COUNT(*) as AttackCount, AVG(VictimCount) as AvgVictims
FROM HighImpactAttacks
GROUP BY Location
ORDER BY AttackCount DESC;

This query achieves the same result as the temporary table example, but the CTE exists only for the duration of this specific query.

#### Comparing Temporary Tables and CTEs

Both techniques have their strengths and are useful in different scenarios:

1.  *Scope and Persistence*
    -   Temporary tables persist for the entire session and can be used across multiple queries.
    -   CTEs are defined and used within a single query.
2.  *Reusability*
    -   Temporary tables can be referenced multiple times in different queries during a session.
    -   CTEs can be referenced multiple times, but only within the query where they're defined.
3.  *Performance*
    -   Temporary tables are actually created and stored, which can be beneficial for large intermediate results that are used multiple times.
    -   CTEs are generally optimized by the query planner and may not be materialized, which can be more efficient for one-time use.
4.  *Syntax and Readability*
    -   CTEs often lead to more readable queries, as the entire logic is contained in a single statement.
    -   Temporary tables may require multiple statements but can make very complex operations more manageable.
5.  *Database Support*
    -   Temporary tables are widely supported across different database systems.
    -   CTE support varies; while most modern databases support them, the specific features and syntax may differ.

#### When to Use Each

Use Temporary Tables when:

-   You need to reference the intermediate results across multiple queries in a session.
-   The intermediate result is large and will be used multiple times.
-   You need to index the intermediate results for performance.

Use CTEs when:

-   You want to improve the readability of a complex query.
-   The intermediate result is used only within a single query.
-   You want to recursively query hierarchical data (recursive CTEs are supported in many databases).

Here's an example of a more complex CTE that demonstrates its power in simplifying queries:

In [48]:
%%sql
-- CTE to calculate monthly statistics
WITH MonthlyStats AS (
    SELECT
        strftime('%Y-%m', Date) as Month,
        ZombieType,
        AVG(VictimCount) as AvgVictims
    FROM ZombieAttacks
    GROUP BY Month, ZombieType
),
-- CTE to find the highest-ranked type for each month
RankedTypes AS (
    SELECT
        Month,
        ZombieType,
        AvgVictims,
        RANK() OVER (PARTITION BY Month ORDER BY AvgVictims DESC) as TypeRank
    FROM MonthlyStats
)
-- Main query
SELECT Month, ZombieType, AvgVictims
FROM RankedTypes
WHERE TypeRank = 1
ORDER BY Month;

 * sqlite:///zombie_attacks.db
Done.


,Month,ZombieType,AvgVictims
0,2023-01,Runner,16.304348
1,2023-02,Runner,15.130435
2,2023-03,Runner,16.500000
3,2023-04,None,16.500000
4,2023-05,Runner,13.285714
5,2023-06,Runner,15.827586
6,2023-07,Runner,15.660000
7,2023-08,Runner,16.275000
8,2023-09,None,17.000000
9,2023-10,Runner,14.217391


This query uses two CTEs to find the zombie type with the highest average victim count for each month. It demonstrates how CTEs can be used to break down a complex query into more understandable steps.


###  Subset of Records

Working with subsets of records is a practical approach to query optimization, particularly useful during the development and testing phases of database operations. This technique involves limiting the amount of data processed in a query, which can significantly reduce execution time and resource consumption. While it may not always be applicable in production environments where complete datasets are necessary, it's an invaluable tool for query development, debugging, and performance tuning.

The primary goal of this approach is to work with a representative sample of your data that's small enough to process quickly, yet large enough to be meaningful for your analysis or testing purposes. This allows for rapid iteration in query development and helps identify potential performance issues before scaling up to the full dataset.

There are several scenarios where working with a subset of records is particularly beneficial:

1.  Query Development: When writing and refining complex queries, testing against a smaller dataset allows for quicker feedback and easier error identification.
2.  Performance Tuning: By running a query on progressively larger subsets of data, you can observe how performance scales and identify potential bottlenecks.
3.  Data Exploration: When initially exploring a new dataset, analyzing a subset can provide quick insights into the data's structure and content.
4.  Testing Data Transformations: When developing ETL (Extract, Transform, Load) processes, working with a subset allows for faster validation of transformation logic.

In SQL, there are several ways to work with subsets of records. Let's explore some of these methods using our Zombie Attacks database:

1.  LIMIT Clause: The LIMIT clause is one of the simplest ways to restrict the number of rows returned by a query. We've seen many examples of this. However, it's important to note that without an ORDER BY clause, the specific 1000 rows returned may not be consistent across multiple executions.
2.  Random Sampling: To get a random sample of records, you can combine ORDER BY RANDOM() with LIMIT. We gave an example of this earlier.
3.  Date-based Subsetting: For time-series data like our Zombie Attacks, you might want to focus on a specific time period. For example:
```sql
SELECT *
    FROM ZombieAttacks
    WHERE Date >= DATE('now', '-30 days')
    LIMIT 1000;
```
This query retrieves up to 1000 attacks from the last 30 days.

When working with subsets of records, it's crucial to ensure that your sample is representative of the full dataset, especially if you're using the results to make broader inferences or decisions. Consider the following:

-   Ensure your subset maintains the general **distribution** of key variables in your full dataset.
-   Include examples of any important **edge cases** or rare events in your subset.
-  For repeatable results, consider using a consistent method of subsetting across your development process.

It's also important to remember that query performance on a subset of data may not always scale linearly to the full dataset. Factors such as index usage, join operations, and memory allocation can behave differently at different data scales. Therefore, while working with subsets is an excellent starting point, it's crucial to validate your final queries against the full dataset or larger subsets before moving to production.

### Query Optimization: Key Points

1.  Query optimization is crucial for improving database performance, especially when dealing with large datasets or high-concurrency systems.
2.  Parameterization
    -   Involves using placeholders for values in SQL queries
    -   Allows the database to cache and reuse execution plans
    -   Improves performance for repetitive queries
    -   Enhances security by preventing SQL injection attacks
3.  Indexing
    -   Creates data structures that speed up data retrieval operations
    -   Particularly useful for columns frequently used in WHERE clauses, JOIN conditions, and ORDER BY statements
    -   Comes with trade-offs: improves read performance but can slow down write operations
    -   Requires careful consideration of query patterns and data distribution
4.  Temporary Tables and CTEs
    -   Used to store intermediate results within a complex query or series of queries
    -   Can simplify complex operations and improve readability
    -   Useful for multi-step data processing, precomputing results, and simplifying complex joins
    -   Should be used judiciously, as they consume additional resources
5.  Working with Subsets of Records
    -   Valuable during query development, testing, and data exploration phases
    -   Can significantly reduce query execution time and resource consumption
    -   Methods include using LIMIT clause, random sampling, and date-based subsetting
    -   Important to ensure the subset is representative of the full dataset
6.  Execution Plans
    -   Provide insight into how the database processes queries
    -   Can be viewed using EXPLAIN commands (syntax varies by database system)
    -   Help identify inefficiencies such as full table scans, inefficient join operations, and lack of index usage
    -   Interpreting execution plans requires understanding of database operations and improves with experience
7.  General Optimization Strategies
    -   Regularly update database statistics to ensure the query optimizer makes informed decisions
    -   Consider the balance between read and write operations when optimizing
    -   Optimization is an iterative process; continually monitor and adjust as data and usage patterns change
    -   Different techniques may be more or less effective depending on the specific database system, data structure, and query patterns
8.  Performance Testing:
    -   Always measure the impact of optimizations; what works in theory may not always provide the expected benefits in practice
    -   Test optimizations with realistic data volumes and concurrency levels
    -   Consider both query execution time and resource utilization (CPU, memory, I/O) when evaluating optimizations

By mastering these query optimization techniques, data scientists can significantly improve the performance of their database operations, leading to more efficient data analysis and better overall system performance.